In [93]:
using Polynomials
using ForwardDiff
using ExponentialAction
using ExponentialUtilities
using LinearAlgebra

In [365]:
A = [0 0.8 0;
     0.8 0 0.2; 
     0 0.2 0]
edge_idx = CartesianIndex(1, 2)
edge_val = A[edge_idx]
A

3×3 Matrix{Float64}:
 0.0  0.8  0.0
 0.8  0.0  0.2
 0.0  0.2  0.0

## Tangent approximation
Here we are computing the derivative for a single edge. We will use the tangent approximation to compute the derivative. This is the approach used in the Akarca paper.

In [392]:
# Points for evaluation
resolution = 0.01
rep_vec = collect(-0.25:resolution:0.25)
reps = [edge_val * (1 + i) for i in rep_vec]

sum_comm = zeros(length(rep_vec))
for (i_rep, rep) in enumerate(reps)
    A_synth = copy(A)
    A_synth[edge_idx] = A_synth[edge_idx[2], edge_idx[1]] = rep
    comm = exp(A_synth)
    sum_comm[i_rep] = sum(comm)
end

x = 1:length(reps)
slope = fit(x, sum_comm, 1)

Polynomial(2.2792638492692374 + 4.8459047353259335*x)

## Finite difference method
Compute the approximate derivative using the finite difference method.
https://en.wikipedia.org/wiki/Finite_difference_method

In [389]:
function approximate_derivative(f, A, delta, edge_idx)
    # Evaluate the function at two nearby points
    A_plus = copy(A)
    A_plus[edge_idx]  = A_plus[edge_idx] +delta

    A_minus = copy(A)
    A_minus[edge_idx]  = A_minus[edge_idx]- delta

    f_plus_delta = sum(f(A_plus))
    f_minus_delta = sum(f(A_minus))

    # Calculate the derivative approximation
    derivative = (f_plus_delta - f_minus_delta) / (2 * delta)

    return derivative
end

# Example usage
f(M) = exp(M)  # Define the function
delta = 0.01  # Small change in input

approximate_derivative(f, A, delta, edge_idx)


2.413251884607348

## Forward Differentiation
Results are derived in the numerator layout, which means that the dimensionality of the input is added to the right.

In [368]:
# construct n x n matrixes of edge combinations
indices = collect(CartesianIndices(A))
index_vec = vec(indices)
index_vec_c = sort(index_vec, by = x -> x[1])
index_vec_r= sort(index_vec, by = x -> x[2])

all_indices = [[[] for _ in 1:length(cartesian_indices)] for _ in 1:length(cartesian_indices)]
for i in 1:size(A, 1)^2
    for j in 1:size(A, 1)^2
        push!(all_indices[i][j], index_vec_r[i], index_vec_c[j])
    end
end

# Any combination of edges that includes a self-loop needs to be removed later
cycle_indices = CartesianIndex[]
for i in 1:size(A, 1)^2
    for j in 1:size(A, 1)^2
        if ((all_indices[i][j][1][1] == all_indices[i][j][1][2])||
            (all_indices[i][j][2][1] == all_indices[i][j][2][2])||
            (all_indices[i][j][1] == all_indices[i][j][2]) ||
            ((all_indices[i][j][1][1] == all_indices[i][j][2][2]) && 
             (all_indices[i][j][1][2] == all_indices[i][j][2][1]))
            )
            push!(cycle_indices, CartesianIndex(i, j))
        end
    end
end

In [390]:
myexp(A) = exponential!(copyto!(similar(A), A), ExpMethodGeneric());
d = ForwardDiff.jacobian(myexp, A)

# we remove all partial derivates that include self loops
# for idx in cycle_indices 
#     d[idx] = 0
# end

d

9×9 Matrix{Float64}:
 1.22423     0.445505    0.0284695   …  0.0284695   0.00557956  0.00022034
 0.445505    1.23135     0.111376       0.00557956  0.0285246   0.00139489
 0.0284695   0.111376    1.11747        0.00022034  0.00139489  0.0276432
 0.445505    0.114098    0.00557956     0.111376    0.0285246   0.00139489
 0.114098    0.4469      0.0285246      0.0285246   0.111725    0.00713115
 0.00557956  0.0285246   0.424582    …  0.00139489  0.00713115  0.106145
 0.0284695   0.00557956  0.00022034     1.11747     0.424582    0.0276432
 0.00557956  0.0285246   0.00139489     0.424582    1.12438     0.106145
 0.00022034  0.00139489  0.0276432      0.0276432   0.106145    1.01381

In [391]:
# we get all partial derivative positions that are non-zero
dx = d[:, findfirst(x -> x == edge_idx, index_vec_c)]
dot(dx, vec(permutedims(exp(A), [2, 1])))

2.4306145412981492

## Forward Differentiation with Jacobian vector product

In [375]:
function jvp(func, primal, tangent)
    g(t) = myexp(primal + t * tangent)
    jvp_result = ForwardDiff.derivative(g, 0.0)
    return jvp_result
end

jvp(myexp, A, exp(A))

3×3 Matrix{Float64}:
 2.59776   2.43061   0.399439
 2.43061   2.69762   0.607654
 0.399439  0.607654  1.09986